In [22]:
#!pip install boto3

### boto3
We'll be using boto3 library. This is the AWS SDK for Python. This SDK allows us to write software that makes use of AWS services. It can look for credentials in multiple places.

Although not recommended, credentials can be directly specified also:

```
bedrock_client = boto3.client(aws_access_key_id="",
                       aws_secret_access_key="",
                       region_name="us-east-1", service_name='bedrock')
```

 For this example, we have stored it as credentials file in .aws folder:

```
[default]
aws_access_key_id = <key_here>
aws_secret_access_key = <secret_here>
```


In [5]:
import boto3
bedrock_client = boto3.client(region_name="us-east-1",
                       service_name='bedrock')
bedrock_client.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'dfa77d8e-8a6b-4aef-949e-94ad09a0266e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 04 May 2024 22:52:14 GMT',
   'content-type': 'application/json',
   'content-length': '23048',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'dfa77d8e-8a6b-4aef-949e-94ad09a0266e'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [6]:
bedrock_client.get_foundation_model(modelIdentifier='amazon.titan-tg1-large')

{'ResponseMetadata': {'RequestId': 'b67e9b5c-d870-45d5-8031-ddd3c423a8c2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 04 May 2024 22:56:09 GMT',
   'content-type': 'application/json',
   'content-length': '383',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b67e9b5c-d870-45d5-8031-ddd3c423a8c2'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
  'modelId': 'amazon.titan-tg1-large',
  'modelName': 'Titan Text Large',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

### To invoke models, we need to go through bedrock-runtime service (and not the client one)

In [8]:
bedrock = boto3.client(region_name="us-east-1",service_name='bedrock-runtime')

### We'll play around with Titan Express model from Amazon

**About model**

Amazon Titan Text Express has a context length of up to 8,000 tokens, making it well-suited for a wide range of advanced, general language tasks such as open-ended text generation and conversational chat, as well as support within Retrieval Augmented Generation (RAG).

```
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

In [9]:
import json

Let us start by asking factual question without any parameters specified:

In [10]:
body = json.dumps({"inputText": "What is the capital of America?"})

In [11]:
response = bedrock.invoke_model(body=body, modelId="amazon.titan-text-express-v1")

In [12]:
print(response)

{'ResponseMetadata': {'RequestId': '00a9e92d-e759-4d2e-a096-bc027288fd60', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 04 May 2024 23:09:57 GMT', 'content-type': 'application/json', 'content-length': '144', 'connection': 'keep-alive', 'x-amzn-requestid': '00a9e92d-e759-4d2e-a096-bc027288fd60', 'x-amzn-bedrock-invocation-latency': '761', 'x-amzn-bedrock-output-token-count': '13', 'x-amzn-bedrock-input-token-count': '7'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x000001B15D5FA800>}


As we can see from above, response is streaming body. Let us read it now:

In [13]:
response_body = json.loads(response.get('body').read())

In [15]:
response_body

{'inputTextTokenCount': 7,
 'results': [{'tokenCount': 13,
   'outputText': '\nWashington, D.C. is the capital of America.',
   'completionReason': 'FINISH'}]}

Print just the output text:

In [16]:
print(response_body["results"][0]["outputText"])


Washington, D.C. is the capital of America.


### Now let us get to question that can have different answers

We'll ask about Big Bang Theory. Model can tell us about actual theory or about the show with same name. We'll first try without any parameters and hence can expect random answer

In [17]:
prompt = "Tell me about big bang theory"

In [18]:
body = json.dumps({"inputText": prompt})
response = bedrock.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
print(response_body["results"][0]["outputText"])


The Big Bang Theory is a scientific theory that proposes that the universe began as a dense and hot state approximately 13.8 billion years ago from a small, compact object known as a singularity. The theory suggests that the universe expanded rapidly during a period known as inflation, and that the various elements and structures we observe in the universe today were formed during this expansion.

The Big Bang Theory is supported by a vast body of scientific evidence, including observations of the cosmic microwave background, the distribution of galaxies, and the laws of physics. However, the theory remains a subject of ongoing research and debate, with some scientists proposing


**It told us about scientific theory. Let us try again. We can get different response about theory itself or about the show**

In [19]:
body = json.dumps({"inputText": prompt})
response = bedrock.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
print(response_body["results"][0]["outputText"])


The Big Bang Theory is a sitcom that aired on CBS from 2007 to 2019. It follows the lives of five characters, Leonard Hofstadter and Sheldon Cooper, who are physicists at Caltech, and their friends Penny, Howard, Raj, and Amy. The show is set in Pasadena, California, and revolves around the characters' scientific and personal lives.

The show is known for its scientific accuracy, as the characters are portrayed as working in the field of theoretical physics, and many of the scientific concepts and theories discussed in the show are real. The show also explores themes of love


**Response was different. It was about the show**

Let us now start **specifying parameters** to make our intentions clear. We want to get **repeatable answers**, so we'll set the value of temperature to zero and topP to low value (as it cannot be zero):

In [20]:
body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0,
        "topP": 0.01,  
        "maxTokenCount": 1024
    }
})
response = bedrock.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
print(response_body["results"][0]["outputText"])


The Big Bang Theory is a sitcom that ran for 12 seasons on CBS. The show follows the lives of five characters, Leonard Hofstadter and Sheldon Cooper, who are physicists at Caltech, and their friends Penny, Howard, and Raj. The show is set in Pasadena, California, and revolves around the characters' personal and professional lives.

The show is known for its scientific accuracy, as the characters are portrayed as working in the field of theoretical physics, and many of the scientific concepts and theories discussed in the show are real. The show also features a lot of humor, with the characters often engaging in witty banter and humorous situations.

One of the key themes of the show is the relationship between Leonard and Sheldon, who are very different in many ways, but share a deep bond of friendship. Leonard is more social and outgoing, while Sheldon is more introverted and analytical. The show explores the challenges and rewards of their friendship, as well as the ways in which th

**Analysis:**

* As we specified maxTokenCount, we got bit longer response
* Response was about the show


In [21]:
body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0,
        "topP": 0.01,  
        "maxTokenCount": 1024
    }
})
response = bedrock.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
print(response_body["results"][0]["outputText"])


The Big Bang Theory is a sitcom that ran for 12 seasons on CBS. The show follows the lives of five characters, Leonard Hofstadter and Sheldon Cooper, who are physicists at Caltech, and their friends Penny, Howard, and Raj. The show is set in Pasadena, California, and revolves around the characters' personal and professional lives.

The show is known for its scientific accuracy, as the characters are portrayed as working in the field of theoretical physics, and many of the scientific concepts and theories discussed in the show are real. The show also features a lot of humor, with the characters often engaging in witty banter and humorous situations.

One of the key themes of the show is the relationship between Leonard and Sheldon, who are very different in many ways, but share a deep bond of friendship. Leonard is more social and outgoing, while Sheldon is more introverted and analytical. The show explores the challenges and rewards of their friendship, as well as the ways in which th

**Analysis:**

* As we specified same maxTokenCount, we got bit longer response
* We specified low values for temperature and topP, we got similar answer